# Data preparation

In [1]:
# Accessing labels from the file
import csv

data = {} # empty dictionary
"""
Format:
key - class name -> 1st row
value - image/file name -> 2nd row
"""

with open('distraced driver\driver_imgs_list.csv') as f:
    reader = csv.reader(f)
    next(reader)
    
    for row in reader:
        key = row[1].lower()
        
        if key in data:
            data[key].append(row[2])
        else:
            data[key] = [row[2]]

In [2]:
data

{'c0': ['img_10206.jpg',
  'img_27079.jpg',
  'img_50749.jpg',
  'img_97089.jpg',
  'img_37741.jpg',
  'img_65697.jpg',
  'img_3866.jpg',
  'img_19098.jpg',
  'img_31885.jpg',
  'img_41423.jpg',
  'img_26907.jpg',
  'img_71304.jpg',
  'img_17470.jpg',
  'img_88793.jpg',
  'img_1367.jpg',
  'img_29348.jpg',
  'img_20878.jpg',
  'img_6461.jpg',
  'img_87811.jpg',
  'img_47410.jpg',
  'img_98046.jpg',
  'img_57583.jpg',
  'img_27214.jpg',
  'img_973.jpg',
  'img_9006.jpg',
  'img_28357.jpg',
  'img_56851.jpg',
  'img_62066.jpg',
  'img_40209.jpg',
  'img_49423.jpg',
  'img_71160.jpg',
  'img_54924.jpg',
  'img_1267.jpg',
  'img_17723.jpg',
  'img_88538.jpg',
  'img_96522.jpg',
  'img_31289.jpg',
  'img_37288.jpg',
  'img_67875.jpg',
  'img_2813.jpg',
  'img_59570.jpg',
  'img_70406.jpg',
  'img_20588.jpg',
  'img_60168.jpg',
  'img_50246.jpg',
  'img_65097.jpg',
  'img_24419.jpg',
  'img_47332.jpg',
  'img_89135.jpg',
  'img_93208.jpg',
  'img_67865.jpg',
  'img_31623.jpg',
  'img_36574.j

In [3]:
# category types
driver_types = list(data.keys())
driver_types

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [4]:
# creating directories
import os

# training and testing directory
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

# subfolders - for each driver types in both training and testing
for driver in driver_types:
    os.mkdir(os.path.join('master_data/training/', driver))
    os.mkdir(os.path.join('master_data/testing/', driver))

In [5]:
"""
Data separation -
1. 80% training data, 20% testing data
2. Copying the data into required location
"""

from shutil import copyfile

split_size = 0.8

for driver, images in data.items():
    train_size = int(split_size * len(images))
    
    train_images = images[:train_size]
    test_images = images[train_size:]
    
    for image in train_images:
        src = os.path.join('distraced driver/new_imgs', image)
        dest = os.path.join('master_data/training', driver, image)
        copyfile(src, dest)

    for image in test_images:
        src = os.path.join('distraced driver/new_imgs', image)
        dest = os.path.join('master_data/testing', driver, image)
        copyfile(src, dest)

# Creating the model

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [7]:
# defining the layers
model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation = 'relu', input_shape = (100, 100, 3)), # input layer
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation = 'relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation = 'relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(1024, activation = 'relu'),
    Dense(10, activation = 'softmax') # output layer
])

In [8]:
# compiling the model
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

# Preparing for training and testing

In [10]:
train_dir = 'master_data/training/'
test_dir = 'master_data/testing/'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (100, 100), class_mode = 'categorical', batch_size = 128)

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (100, 100), class_mode = 'categorical', batch_size = 128)

Found 17462 images belonging to 10 classes.
Found 6990 images belonging to 10 classes.


In [11]:
es = EarlyStopping(monitor = 'val_acc', patience = 2, min_delta = 0.01)

# Training and testing the model

In [12]:
model.fit(train_generator, epochs = 100, verbose = 1, validation_data = test_generator, callbacks = [es])

Epoch 1/100
137/137 [==============================] - 373s 3s/step - loss: 1.4730 - acc: 0.5691 - val_loss: 0.4189 - val_acc: 0.8690
Epoch 2/100
137/137 [==============================] - 153s 1s/step - loss: 0.2962 - acc: 0.9077 - val_loss: 0.2014 - val_acc: 0.9325
Epoch 3/100
137/137 [==============================] - 153s 1s/step - loss: 0.1711 - acc: 0.9466 - val_loss: 0.1297 - val_acc: 0.9594
Epoch 4/100
137/137 [==============================] - 155s 1s/step - loss: 0.1186 - acc: 0.9623 - val_loss: 0.0699 - val_acc: 0.9755
Epoch 5/100
137/137 [==============================] - 153s 1s/step - loss: 0.3537 - acc: 0.9029 - val_loss: 0.2965 - val_acc: 0.9030
Epoch 6/100
137/137 [==============================] - 154s 1s/step - loss: 0.1308 - acc: 0.9591 - val_loss: 0.0723 - val_acc: 0.9783
